In [1]:
import spacy
import gensim
import os
import pandas as pd
from gensim.models import KeyedVectors
from joblib import Parallel, delayed
import itertools
import numpy as np
import pickle

construct dataset

In [2]:
def get_files(dir_path):
    ngs = ['LICENSE.txt']
    files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f)) and f not in ngs]
    return files

def get_lines(file_path):
    with open(file_path, 'r') as f:
        lines = [l.strip() for l in f.readlines()]
    return [l for l in lines if len(l) > 0]

p = '/data/livedoor/text/'
medium = [d for d in os.listdir(p) if os.path.isdir(os.path.join(p, d))]

dataset = []

for media in medium:
    dir_path = os.path.join(p, media)
    files = get_files(dir_path)
    
    for file in files:
        
        file_path = os.path.join(dir_path, file)
        lines = get_lines(file_path)
        
        url, timestamp, title, text = lines[0], lines[1], lines[2], ''.join(lines[3:])
        dataset.append((media, url, timestamp, title, text))

dataset = pd.DataFrame(dataset, columns=['media', 'url', 'timestamp', 'title', 'text'])
dataset = dataset[dataset.index!=6031]
dataset.head()

,media,url,timestamp,title,text
0,livedoor-homme,http://news.livedoor.com/article/detail/5274747/,2011-01-25T09:00:00+0900,プロに聞く“合コンの極意”（後編） 合コンアナリスト水谷麻衣に聞く、勝ち合コンの鉄則とは,前回の「プロに聞く“合コンの極意”（前編） 合コンアナリスト水谷麻衣に聞く、合コンの勝ちパタ...
1,livedoor-homme,http://news.livedoor.com/article/detail/5736745/,2011-09-02T11:00:00+0900,「会社の人が必要以上にプライベートにも関わってくる」- 辛口説教部屋 vol.14,「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽き...
2,livedoor-homme,http://news.livedoor.com/article/detail/5825784/,2011-10-21T11:00:00+0900,年収1000万円のビジネスマンに聞いた「自分の商品価値を高めるための努力」 - 年収1000...,こんにちは、「ビズリーチ年収1000万円研究所」所長の佐藤和男です。この研究所では、年収10...
3,livedoor-homme,http://news.livedoor.com/article/detail/5642391/,2011-06-20T09:30:00+0900,Hennessy（ヘネシー）×livedoorブロガーイベント開催,6月7日、表参道のカフェバー「MERCER CAFE TERRACE HOUSE」でHenn...
4,livedoor-homme,http://news.livedoor.com/article/detail/5769394/,2011-11-18T11:00:00+0900,「新入社員との間に、溝が出来てしまって悩んでいます」 - 辛口説教部屋 vol.47,「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽き...


tokenize

In [3]:
nlp = spacy.load('ja_ginza')

def split_list(ls, step):
    n = len(ls)
    for i in range(0, n, step):
        yield ls[i:i+step]

def tokenize(batch):
    return [[str(token) for token in nlp(text)] for text in batch]

In [4]:
%%time
ls = list(split_list(dataset['text'].to_list(), 100))
r = Parallel(n_jobs=-1)([delayed(tokenize)(batch) for batch in ls])
r = list(itertools.chain.from_iterable(r))
dataset.loc[:, 'tokens'] = r

# p2.xlarge
# CPU times: user 2.43 s, sys: 378 ms, total: 2.8 s
# Wall time: 4min 50s

# m5.4xlarge
# CPU times: user 1.88 s, sys: 402 ms, total: 2.29 s
# Wall time: 58.4 s

CPU times: user 1.82 s, sys: 406 ms, total: 2.23 s
Wall time: 57.4 s


embedding

In [5]:
%%time
wv = KeyedVectors.load_word2vec_format('/data/chive/chive-1.2-mc5/chive-1.2-mc5.txt')
wv
# p2.xlarge
# CPU times: user 14min 52s, sys: 10.3 s, total: 15min 3s
# Wall time: 15min 3s

# m5.4xlarge
# CPU times: user 12min 26s, sys: 8.34 s, total: 12min 35s
# Wall time: 12min 35s

CPU times: user 11min 59s, sys: 8.45 s, total: 12min 8s
Wall time: 12min 8s


In [6]:
%%time
def embed(tokens):
    embeddings = []
    for t in tokens:
        try:
            embeddings.append(list(wv[t]))
        except:
            pass
    return embeddings

# parquet は np.array 出力できないため list 形式
dataset.loc[:, 'embeddings'] = dataset['tokens'].apply(lambda tokens: embed(tokens))
dataset.head()

# p2.xlarge
# CPU times: user 2min 13s, sys: 27.8 s, total: 2min 41s
# Wall time: 2min 41s

CPU times: user 1min 43s, sys: 21.9 s, total: 2min 5s
Wall time: 2min 5s


,media,url,timestamp,title,text,tokens,embeddings
0,livedoor-homme,http://news.livedoor.com/article/detail/5274747/,2011-01-25T09:00:00+0900,プロに聞く“合コンの極意”（後編） 合コンアナリスト水谷麻衣に聞く、勝ち合コンの鉄則とは,前回の「プロに聞く“合コンの極意”（前編） 合コンアナリスト水谷麻衣に聞く、合コンの勝ちパタ...,"[前回, の, 「, プロ, に, 聞く, “, 合コン, の, 極意, ”, （, 前編,...","[[-0.111276, -0.047093, -0.026511, 0.046474, -..."
1,livedoor-homme,http://news.livedoor.com/article/detail/5736745/,2011-09-02T11:00:00+0900,「会社の人が必要以上にプライベートにも関わってくる」- 辛口説教部屋 vol.14,「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽き...,"[「, 3, 年, で, 転職, は, 早, すぎる, ？, 」, 「, 将来, が, 見え...","[[0.017996, 0.079715, 0.094203, -0.175814, 0.0..."
2,livedoor-homme,http://news.livedoor.com/article/detail/5825784/,2011-10-21T11:00:00+0900,年収1000万円のビジネスマンに聞いた「自分の商品価値を高めるための努力」 - 年収1000...,こんにちは、「ビズリーチ年収1000万円研究所」所長の佐藤和男です。この研究所では、年収10...,"[こんにちは, 、, 「, ビズリーチ, 年収, 1000万, 円, 研究, 所, 」, 所...","[[-0.018621, 0.027331, 0.092579, -0.038892, -0..."
3,livedoor-homme,http://news.livedoor.com/article/detail/5642391/,2011-06-20T09:30:00+0900,Hennessy（ヘネシー）×livedoorブロガーイベント開催,6月7日、表参道のカフェバー「MERCER CAFE TERRACE HOUSE」でHenn...,"[6, 月, 7, 日, 、, 表参道, の, カフェバー, 「, MERCER, CAFE...","[[0.097441, -0.056396, 0.064514, 0.004136, -0...."
4,livedoor-homme,http://news.livedoor.com/article/detail/5769394/,2011-11-18T11:00:00+0900,「新入社員との間に、溝が出来てしまって悩んでいます」 - 辛口説教部屋 vol.47,「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽き...,"[「, 3, 年, で, 転職, は, 早, すぎる, ？, 」, 「, 将来, が, 見え...","[[0.017996, 0.079715, 0.094203, -0.175814, 0.0..."


In [7]:
%%time
# Kernel dies -> add memory
dataset.to_parquet('./data/word_embeddings.parquet.gz', compression='gzip')

CPU times: user 7min 20s, sys: 21.2 s, total: 7min 41s
Wall time: 7min 39s


In [ ]:
# %%time
# # pickle
# with open('./data/word_embeddings.pickle', 'wb') as f:
#     pickle.dump(dataset, f)

In [ ]:
# csv
# corpus.to_csv('./data/word_embeddings.csv', index=False)
# list を保持できないため csv 形式での保存は不採用